In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import tqdm

import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import copy
# import multiprocessing as mp
from torch.multiprocessing import Pipe

import gym

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

device = torch.device("cpu")

import caviar_tools
from beamselect_env import BeamSelectionEnv

reward_type = 'test'    # 'test' or 'train'
# epi = [0,9] #[start,end] 
epi_val = [500,509]

# gym_env_train = BeamSelectionEnv(epi,reward_type)

# gym_env_ind = list()
# for i in range(epi[0],epi[1]+1):
#     gym_env_ind.append(BeamSelectionEnv([i,i],reward_type))

gym_env_val = BeamSelectionEnv(epi_val)

# n_steps_epi = list()
n_steps_epi_val = list()
# for i in range(epi[0],epi[1]+1):
#     n_steps_epi.append(caviar_tools.linecount([i,i]))

for i in range(epi_val[0],epi_val[1]+1):
    n_steps_epi_val.append(caviar_tools.linecount([i,i]))

# n_steps = sum(n_steps_epi)
n_steps_val = sum(n_steps_epi_val)

/home/sundesh/Documents/git/ITU-Challenge/torch_rl/lib/python3.6/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [2]:
from agents_cur import *
from utils_cur import *

import numpy as np
import pickle

In [3]:
train_method = 'ICM'
env_id = None #BreakoutNoFrameskip-v4
env_type = 'beamselect'
env = gym_env_val

In [4]:
HISTORY_SIZE = 16
STATES_USED = 13

input_size = [HISTORY_SIZE,STATES_USED]  
output_size = 192 #64*3

In [5]:
model_path = './model_curiosity'
icm_path = './icm_curiosity'

In [6]:
lam = 0.95
num_worker = 1

num_step = int(128)

ppo_eps = float(0.1)
epoch = int(3)
mini_batch = int(8)
BATCH_SIZE = int(num_step * num_worker / mini_batch) #16
learning_rate = float(1e-4)
entropy_coef = float(0.001)
gamma = float(0.99)
eta = float(1)

clip_grad_norm = float(0.5)

pre_obs_norm_step = int(10)#int(10000)

HISTORY_SIZE = 16
STATES_USED = 13

sticky_action = False
life_done = False

In [7]:
obs_rms = RunningMeanStd(shape=(1, HISTORY_SIZE, 1, STATES_USED))

In [8]:
agent = ICMAgent

agent = agent(
        input_size,
        output_size,
        num_worker,
        num_step,
        gamma,
        lam=lam,
        learning_rate=learning_rate,
        ent_coef=entropy_coef,
        clip_grad_norm=clip_grad_norm,
        epoch=epoch,
        batch_size=BATCH_SIZE,
        ppo_eps=ppo_eps,
        eta=eta,
        use_cuda=False,
        use_gae=False,
        use_noisy_net=False
)

In [9]:
states = np.zeros([1, HISTORY_SIZE, 1,STATES_USED])

In [10]:
agent.model.load_state_dict(torch.load(model_path))
agent.icm.load_state_dict(torch.load(icm_path))

<All keys matched successfully>

In [11]:
Transition = namedtuple('Transition',
                        ('state'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size:int=BATCH_SIZE):
        return random.sample(self.memory, BATCH_SIZE)

    def __len__(self):
        return len(self.memory)

In [12]:
history = ReplayMemory(HISTORY_SIZE)

for i in range(HISTORY_SIZE):
    history.push(np.zeros((STATES_USED, )))

In [13]:
def run(action):
    s, reward, done, info = env.step([action//64, action%64])
    # print(type(s), s.shape)
    history.push(s.astype(np.float))
    
    return [np.array(history.sample(BATCH_SIZE)), reward, done, done, reward]

In [14]:
f = open('obs_rms.pkl', 'rb')
obs_rms = pickle.load(f)

In [15]:
steps = 0
rall = 0
rd = False
intrinsic_reward_list = []
while steps < n_steps_val:
    steps += 1
    actions, value, policy = agent.get_action((states - obs_rms.mean) / np.sqrt(obs_rms.var))

    next_states, rewards, dones, real_dones, log_rewards, next_obs = [], [], [], [], [], []
    for action in actions:
        s, r, d, rd, lr = run(action)
        rall += r
        next_states.append(s)
#         next_obs = s[3, :, :].reshape([1, 1, 1,STATES_USED])

    # total reward = int reward + ext Reward
#     intrinsic_reward = agent.compute_intrinsic_reward(next_obs)
#     intrinsic_reward_list.append(intrinsic_reward)
    next_states = np.stack(next_states)
    states = next_states[:, :, :, :]

#     if rd:
#         intrinsic_reward_list = (intrinsic_reward_list - np.mean(intrinsic_reward_list)) / np.std(
#             intrinsic_reward_list)
#         with open('int_reward', 'wb') as f:
#             pickle.dump(intrinsic_reward_list, f)
#         steps = 0
#         rall = 0

print(f"Total Reward: {rall}, Avg Reward: {rall/steps}")

KeyboardInterrupt: 